# Generador de Contenido Educativo con LLM

In [ ]:
!apt-get update
!apt-get install -y pandoc texlive-xetex
!pip install markdown2 pypandoc google-generativeai
!pip install pdfplumber
!pip install pytesseract pdf2image
!pip install DOCX PyPDF2
!pip install pypandoc python-docx pdfplumber pdf2image pytesseract
!apt-get update
!apt-get install -y pandoc texlive-xetex
!pip install markdown2 pypandoc google-generativeai

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,988 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,533 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 4,905 kB in 1s (5,840 kB/s)
Reading package lists... Done
W: Skipping acquire of

In [ ]:
# Course content
course_content = """
Contenido del curso Cálculo diferencial.
Unidades:
1. Introducción a la derivada
2. Derivada de la función compuesta
3. Aplicaciones de la derivada en problemas de razón de cambio

Autor: Juan David Ospina Arango
Profesor Asociado
Departamento de Ciencias de la Computación y de la Decisión
Universidad Nacional de Colombia
"""

# Your API key
api_key = "Mi_API"  # Replace with your real Google API key

# Execute the code - everything in one cell
import os
import time
import json
import logging
import hashlib
from datetime import datetime
from pathlib import Path
import google.generativeai as genai
import markdown2
import pypandoc

# Configure basic logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s - %(message)s')
logger = logging.getLogger("eduagent")

class GeminiClient:
    """Client to interact with Google Gemini API for content generation."""

    def __init__(self, api_key, model="gemini-2.0-flash-001", temperature=0.4, max_tokens=4000,
                 cache_dir="./data/cache/gemini", use_cache=True, max_retries=3):
        self.api_key = api_key
        genai.configure(api_key=self.api_key)
        self.model = model
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.max_retries = max_retries
        self.use_cache = use_cache
        self.total_tokens_used = 0
        self.total_requests = 0
        self.total_cost = 0.0
        self.request_history = []
        self.cache_dir = Path(cache_dir)
        self.cache_dir.mkdir(parents=True, exist_ok=True)
        self.generation_config = {
            "temperature": temperature,
            "max_output_tokens": max_tokens,
            "top_p": 0.9,
            "top_k": 40
        }

    def _calculate_cost(self, prompt_tokens, completion_tokens):
        model_prices = {
            "gemini-2.0-flash-001": {"prompt": 0.0001, "completion": 0.0002},
            "gemini-1.5-flash-001": {"prompt": 0.0001, "completion": 0.0002},
            "gemini-1.5-pro-001": {"prompt": 0.0005, "completion": 0.0015}
        }
        price_info = model_prices.get(self.model, {"prompt": 0.0001, "completion": 0.0002})
        prompt_cost = (prompt_tokens / 1000) * price_info["prompt"]
        completion_cost = (completion_tokens / 1000) * price_info["completion"]
        return prompt_cost + completion_cost

    def generate_content(self, prompt, model=None, temperature=None, max_tokens=None, use_cache=None):
        effective_model = model or self.model
        effective_temp = temperature if temperature is not None else self.temperature
        effective_max_tokens = max_tokens or self.max_tokens

        try:
            model_instance = genai.GenerativeModel(effective_model)
            generation_config = {
                "temperature": effective_temp,
                "max_output_tokens": effective_max_tokens,
                "top_p": 0.9,
                "top_k": 40
            }

            result = model_instance.generate_content(prompt, generation_config=generation_config)

            prompt_tokens = len(prompt) // 4
            completion_tokens = len(result.text) // 4

            self.total_requests += 1
            self.total_tokens_used += prompt_tokens + completion_tokens
            cost = self._calculate_cost(prompt_tokens, completion_tokens)
            self.total_cost += cost

            return result.text, {
                "from_cache": False,
                "model": effective_model,
                "prompt_tokens": prompt_tokens,
                "completion_tokens": completion_tokens,
                "cost": cost
            }

        except Exception as e:
            logger.error(f"Error generating content: {e}")
            return "", {"error": str(e)}

    def generate_with_template(self, template, variables, model=None, temperature=None, max_tokens=None, use_cache=None):
        filled_prompt = template
        for var_name, var_value in variables.items():
            filled_prompt = filled_prompt.replace(f"{{{var_name}}}", str(var_value))

        return self.generate_content(
            prompt=filled_prompt,
            model=model,
            temperature=temperature,
            max_tokens=max_tokens,
            use_cache=use_cache
        )

    def get_usage_stats(self):
        return {
            "total_requests": self.total_requests,
            "total_tokens_used": self.total_tokens_used,
            "total_cost_usd": self.total_cost,
            "models_used": list(set(req.get("model", self.model) for req in self.request_history)) if self.request_history else [self.model],
            "average_tokens_per_request": self.total_tokens_used / max(1, self.total_requests),
            "average_cost_per_request": self.total_cost / max(1, self.total_requests)
        }

class CourseContentGenerator:
    """Generate educational content for courses using LLM."""

    def __init__(self, llm_client, output_dir="output"):
        self.llm_client = llm_client
        self.output_dir = Path(output_dir)
        self.output_dir.mkdir(parents=True, exist_ok=True)
# Template for course materials
        self.templates = {
            "class_notes": """
[CONTEXTO DEL CURSO]
Título del curso: {title}
Descripción: {description}
Unidad actual: {unit_title}
Tema específico: {topic}
Objetivos de aprendizaje relacionados: {objectives}
[INSTRUCCIÓN]
Actúa como un profesor universitario experto en {subject_area} creando notas de clase detalladas sobre "{topic}" para estudiantes universitarios de nivel {course_level}.
[ESTRUCTURA REQUERIDA]
1. Introducción conceptual (1-2 párrafos)
2. Fundamentos teóricos con definiciones precisas
3. Desarrollo detallado de conceptos clave, organizados jerárquicamente
4. Explicación de aplicaciones prácticas y ejemplos ilustrativos
5. Conexiones con otros temas del curso
6. Resumen de puntos principales y conclusiones
7. Preguntas de reflexión para los estudiantes
[REQUISITOS DE CONTENIDO]
- Incluye definiciones rigurosas y académicamente precisas
- Incorpora ejemplos claros que ilustren conceptos abstractos
- Utiliza un lenguaje académico pero accesible para estudiantes universitarios
- Incluye referencias a conceptos previos de la unidad cuando sea relevante
- Mantén una profundidad académica apropiada para el nivel universitario
- Incorpora la terminología específica del campo: {key_terms}
- Evita simplificaciones excesivas o generalizaciones incorrectas
- Asegúrate de cubrir todos los aspectos del tema mencionados en el programa del curso
[FORMATO]
- Utiliza encabezados y subencabezados para organizar el contenido
- Incluye listas con viñetas para puntos importantes
- Añade énfasis en los conceptos clave
- Mantén una extensión aproximada de {word_count} palabras
- Estructura los párrafos de forma clara y coherente
Por favor, genera notas de clase completas siguiendo estas directrices.
""",
            "exercises": """
[CONTEXTO DEL CURSO]
Título del curso: {title}
Descripción: {description}
Unidad actual: {unit_title}
Tema específico: {topic}
Objetivos de aprendizaje relacionados: {objectives}
[INSTRUCCIÓN]
Actúa como un profesor universitario experto en {subject_area} creando ejercicios prácticos sobre "{topic}" para estudiantes universitarios de nivel {course_level}.
[REQUISITOS]
- Genera {exercise_count} ejercicios con diferentes niveles de dificultad (básico, intermedio, avanzado)
- Cada ejercicio debe incluir una explicación clara del problema
- Proporciona la solución detallada paso a paso para cada ejercicio
- Los ejercicios deben ayudar a comprender los conceptos teóricos del tema
- Incluye aplicaciones prácticas y casos de estudio relevantes
[FORMATO]
- Organiza los ejercicios por nivel de dificultad
- Numera cada ejercicio claramente
- Utiliza un formato estructurado para las soluciones
- Incluye consejos o pistas donde sea apropiado
Por favor, genera los ejercicios prácticos siguiendo estas directrices.
"""
        }

    def parse_course_content(self, content):
        """Parse course content from text format."""
        lines = content.strip().split('\n')
        course_info = {}

        # Extract basic course information
        title_line = next((line for line in lines if line.startswith("Contenido del curso")), "")
        if title_line:
            course_info["title"] = title_line.replace("Contenido del curso", "").strip()

        # Extract units
        units = []
        current_unit = None

        for line in lines:
            line = line.strip()
            if not line:
                continue

            # Check if this is a unit heading (assumes "1. Unit Name" format)
            if line[0].isdigit() and line[1:3] in ['. ', '- ']:
                if current_unit:
                    units.append(current_unit)
                unit_number = line[0]
                unit_name = line[3:].strip()
                current_unit = {
                    "number": unit_number,
                    "name": unit_name,
                    "topics": []
                }

        # Add the last unit if exists
        if current_unit:
            units.append(current_unit)

        course_info["units"] = units

        # Authorship information
        author_section = next((i for i, line in enumerate(lines) if line.startswith("Autor:")), -1)
        if author_section >= 0:
            author_lines = []
            for i in range(author_section, min(author_section + 5, len(lines))):
                if lines[i].strip():
                    author_lines.append(lines[i].strip())
            course_info["author"] = " ".join(author_lines)

        return course_info

    def generate_course_materials(self, course_info, material_type="class_notes"):
        """Generate course materials based on course info."""
        logger.info(f"Generating {material_type} for course: {course_info.get('title', 'Unknown')}")

        template = self.templates.get(material_type)
        if not template:
            logger.error(f"No template found for material type: {material_type}")
            return None

        materials = {}
        units = course_info.get("units", [])

        for unit in units:
            unit_number = unit.get("number")
            unit_name = unit.get("name")
            unit_key = f"unit_{unit_number}"

            logger.info(f"Generating materials for Unit {unit_number}: {unit_name}")

            # Prepare variables for template
            variables = {
                "title": course_info.get("title", ""),
                "description": course_info.get("description", "Course description not available"),
                "unit_title": unit_name,
                "topic": unit_name,
                "objectives": "Understand core concepts and applications of this unit",
                "subject_area": course_info.get("title", "").split()[0],  # Simple extraction of subject
                "course_level": "pregrado",
                "key_terms": "",
                "word_count": "1500",
                "exercise_count": "5"
            }

            # Generate content
            content, metadata = self.llm_client.generate_with_template(
                template=template,
                variables=variables
            )

            materials[unit_key] = content

            # Log generation
            tokens = metadata.get("prompt_tokens", 0) + metadata.get("completion_tokens", 0)
            logger.info(f"Generated {material_type} for Unit {unit_number}. Tokens: {tokens}")

        return materials

    def save_materials(self, materials, course_info, material_type, format="markdown"):
        """Save generated materials to files."""
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        material_dir = self.output_dir / material_type / timestamp
        material_dir.mkdir(parents=True, exist_ok=True)

        saved_files = []

        # Save individual unit materials
        for unit_key, content in materials.items():
            # Clean filename
            filename = f"{unit_key}_{material_type}"

            # Save as markdown
            md_path = material_dir / f"{filename}.md"
            with open(md_path, 'w', encoding='utf-8') as f:
                f.write(content)
            saved_files.append(md_path)

            # Convert to PDF if requested
            if format == "pdf":
                try:
                    pdf_path = material_dir / f"{filename}.pdf"
                    html = markdown2.markdown(content)
                    extra_args = ['--pdf-engine=xelatex', '-V', 'mainfont=Latin Modern Roman']
                    pypandoc.convert_text(html, 'pdf', format='html', outputfile=str(pdf_path), extra_args=extra_args)
                    saved_files.append(pdf_path)
                except Exception as e:
                    logger.error(f"Error converting to PDF: {e}")

        # Save combined materials
        if len(materials) > 1:
            combined_content = f"# {course_info.get('title', 'Course')} - {material_type.replace('_', ' ').title()}\n\n"
            combined_content += f"Author: {course_info.get('author', 'Unknown')}\n\n"

            for unit_key, content in materials.items():
                unit_number = unit_key.split('_')[1]
                unit_name = next((u.get('name', '') for u in course_info.get('units', []) if u.get('number') == unit_number), '')
                combined_content += f"## Unit {unit_number}: {unit_name}\n\n"
                combined_content += content
                combined_content += "\n\n---\n\n"

            # Save combined markdown
            combined_path = material_dir / f"{course_info.get('title', 'course')}_{material_type}.md"
            with open(combined_path, 'w', encoding='utf-8') as f:
                f.write(combined_content)
            saved_files.append(combined_path)

            # Convert to PDF if requested
            if format == "pdf":
                try:
                    pdf_path = material_dir / f"{course_info.get('title', 'course')}_{material_type}.pdf"
                    html = markdown2.markdown(combined_content)
                    extra_args = ['--pdf-engine=xelatex', '-V', 'mainfont=Latin Modern Roman']
                    pypandoc.convert_text(html, 'pdf', format='html', outputfile=str(pdf_path), extra_args=extra_args)
                    saved_files.append(pdf_path)
                except Exception as e:
                    logger.error(f"Error converting combined content to PDF: {e}")

        logger.info(f"Saved {len(saved_files)} files to {material_dir}")
        return saved_files

def generate_course_materials(course_content=None, output_dir="output", model="gemini-2.0-flash-001", output_format="markdown", api_key=None):
    """Generate educational materials - designed to be called directly."""
    if not api_key:
        raise ValueError("API key is required")

    # If no course content provided, use a default example
    if not course_content:
        course_content = """
Contenido del curso Cálculo diferencial.
Unidades:
1. Introducción a la derivada
2. Derivada de la función compuesta
3. Aplicaciones de la derivada en problemas de razón de cambio

Autor: Juan David Ospina Arango
Profesor Asociado
Departamento de Ciencias de la Computación y de la Decisión
Universidad Nacional de Colombia
"""

    try:
        # Initialize Gemini client with provided API key
        gemini_client = GeminiClient(
            api_key=api_key,
            model=model,
            temperature=0.4,
            max_tokens=8192,
            use_cache=True
        )

        # Initialize content generator
        generator = CourseContentGenerator(
            llm_client=gemini_client,
            output_dir=output_dir
        )

        # Parse course content
        course_info = generator.parse_course_content(course_content)

        # Generate class notes
        class_notes = generator.generate_course_materials(
            course_info=course_info,
            material_type="class_notes"
        )

        # Save class notes
        notes_files = generator.save_materials(
            materials=class_notes,
            course_info=course_info,
            material_type="class_notes",
            format=output_format
        )

        # Generate exercises
        exercises = generator.generate_course_materials(
            course_info=course_info,
            material_type="exercises"
        )

        # Save exercises
        exercise_files = generator.save_materials(
            materials=exercises,
            course_info=course_info,
            material_type="exercises",
            format=output_format
        )

        # Print usage statistics
        usage_stats = gemini_client.get_usage_stats()
        logger.info(f"Total requests: {usage_stats['total_requests']}")
        logger.info(f"Total tokens used: {usage_stats['total_tokens_used']}")
        logger.info(f"Estimated cost: ${usage_stats['total_cost_usd']:.4f}")

        logger.info(f"Materials generated successfully. Check the {output_dir} directory for files.")
        return notes_files, exercise_files, usage_stats

    except Exception as e:
        logger.error(f"Error in content generation: {e}")
        return None, None, {"error": str(e)}

# Run the generator with your course content and API key
notes_files, exercise_files, usage_stats = generate_course_materials(
    course_content=course_content,
    output_dir="output_materials",
    model="gemini-2.0-flash-001",
    output_format="markdown",
    api_key=api_key  # Make sure to replace this with your real API key
)

# Print results
if notes_files and exercise_files:
    print(f"\nSUCCESS! Generated {len(notes_files)} note files and {len(exercise_files)} exercise files")
    print(f"Total cost: ${usage_stats['total_cost_usd']:.4f}")
    print(f"Check the output_materials directory for your files!")
else:
    print("\nERROR: Failed to generate materials.")
    print("Error details:", usage_stats.get("error", "Unknown error"))


SUCCESS! Generated 4 note files and 4 exercise files
Total cost: $0.0024
Check the output_materials directory for your files!


In [ ]:
# Course content
course_content = """
Contenido del curso Analisis y Diseño de algoritmos.
Unidades:
1. Introducción a Big O
2. Algoritmos dinamicos
3. Divide y venceras

Autor: Julian Moreno
Departamento de Ciencias de la Computación y de la Decisión
Universidad Nacional de Colombia
"""

# Your API key
api_key = "Mi_API"  # Replace with your real Google API key

# Run the generator with your course content and API key
notes_files, exercise_files, usage_stats = generate_course_materials(
    course_content=course_content,
    output_dir="output_materials",
    model="gemini-2.0-flash-001",
    output_format="markdown",
    api_key=api_key  # Make sure to replace this with your real API key
)

# Print results
if notes_files and exercise_files:
    print(f"\nSUCCESS! Generated {len(notes_files)} note files and {len(exercise_files)} exercise files")
    print(f"Total cost: ${usage_stats['total_cost_usd']:.4f}")
    print(f"Check the output_materials directory for your files!")
else:
    print("\nERROR: Failed to generate materials.")
    print("Error details:", usage_stats.get("error", "Unknown error"))


SUCCESS! Generated 4 note files and 4 exercise files
Total cost: $0.0033
Check the output_materials directory for your files!


In [ ]:
# Extensión de plantillas para los materiales adicionales requeridos
additional_templates = {
    "discussion_questions": """
[CONTEXTO DEL CURSO]
Título del curso: {title}
Descripción: {description}
Unidad actual: {unit_title}
Tema específico: {topic}
Objetivos de aprendizaje relacionados: {objectives}
[INSTRUCCIÓN]
Actúa como un profesor universitario experto en {subject_area} creando preguntas de discusión para "{topic}" para estudiantes universitarios de nivel {course_level}.
[REQUISITOS]
- Genera 5-7 preguntas de discusión que fomenten el pensamiento crítico
- Las preguntas deben explorar diferentes aspectos del tema
- Incluye preguntas que conecten con aplicaciones prácticas o casos reales
- Fomenta la reflexión sobre implicaciones éticas cuando sea relevante
- Incluye preguntas que relacionen este tema con otros del curso
[FORMATO]
- Organiza las preguntas en secciones temáticas
- Para cada pregunta, incluye una breve nota sobre los aspectos que busca explorar
- Añade sugerencias para facilitar la discusión en clase
Por favor, genera preguntas de discusión siguiendo estas directrices.
""",

    "learning_objectives": """
[CONTEXTO DEL CURSO]
Título del curso: {title}
Descripción: {description}
Unidad actual: {unit_title}
Tema específico: {topic}
[INSTRUCCIÓN]
Actúa como un diseñador curricular experto creando objetivos de aprendizaje para el tema "{topic}" en un curso universitario de {subject_area} de nivel {course_level}.
[REQUISITOS]
- Utiliza la taxonomía de Bloom para diferentes niveles cognitivos
- Genera 5-8 objetivos claros y medibles
- Incluye objetivos para conocimiento, comprensión, aplicación, análisis y evaluación
- Los objetivos deben ser específicos y alcanzables en el contexto del curso
- Asegúrate que sean relevantes para las competencias del área de {subject_area}
[FORMATO]
- Organiza los objetivos por nivel cognitivo
- Utiliza verbos de acción específicos al inicio de cada objetivo
- Incluye una breve explicación de cómo cada objetivo contribuye al dominio del tema
- Añade sugerencias de actividades que ayuden a lograr cada objetivo
Por favor, genera objetivos de aprendizaje siguiendo estas directrices.
""",

    "suggested_readings": """
[CONTEXTO DEL CURSO]
Título del curso: {title}
Descripción: {description}
Unidad actual: {unit_title}
Tema específico: {topic}
Objetivos de aprendizaje relacionados: {objectives}
[INSTRUCCIÓN]
Actúa como un bibliotecario académico experto en {subject_area} recomendando recursos de lectura para el tema "{topic}" para estudiantes universitarios de nivel {course_level}.
[REQUISITOS]
- Recomienda 8-10 recursos académicos relevantes (libros, artículos, recursos web)
- Incluye tanto recursos fundamentales clásicos como literatura reciente
- Para cada recurso académico, proporciona: autor, título, año, editorial/revista y una breve descripción
- Categoriza las lecturas (básicas, complementarias, avanzadas)
- Incluye al menos 2-3 recursos en español y el resto en inglés
- Añade comentarios sobre la relevancia específica de cada recurso para el tema
[FORMATO]
- Organiza los recursos por categoría y relevancia
- Proporciona referencias en formato APA
- Incluye una breve justificación para cada recurso
- Si es posible, sugiere capítulos o secciones específicas
Por favor, genera recomendaciones de lecturas siguiendo estas directrices.
"""
}

# Actualiza las plantillas existentes en tu clase CourseContentGenerator
# Esta celda debe ejecutarse después de haber definido la clase CourseContentGenerator
def update_course_generator_templates():
    # Referencia al generador de contenido global o crea uno nuevo
    try:
        # Intenta acceder a un generador ya definido en el espacio global
        global generator
        if 'generator' in globals():
            generator.templates.update(additional_templates)
            print("Plantillas actualizadas en el generador existente.")
        else:
            # Si no existe, crea un mensaje informativo
            print("No se encontró el generador. Ejecuta primero la celda que define CourseContentGenerator.")
    except Exception as e:
        print(f"Error al actualizar plantillas: {e}")

In [ ]:
def extract_course_content(response):
    """
    Extrae y retorna el contenido generado en formato de código Python
    a partir del objeto de respuesta del modelo.
    """
    # Intentar acceder como objeto con atributos

    try:
        return response.candidates[0].content.parts[0].text

    except AttributeError:
        # Si falla, intentar acceder como diccionario
        try:
            print("Modelo Gemini inicializado.2")
            return response['candidates'][0]['content']['parts'][0]['text']
        except (KeyError, TypeError):
            print("Modelo Gemini inicializado.3")
            raise ValueError("No se pudo extraer el contenido de la respuesta.")

def generate_course_content(custom_prompt):
    """
    Llama al modelo Gemini con el prompt proporcionado y retorna el contenido
    en el formato solicitado.
    """

    # Inicializa el modelo Gemini
    model = genai.GenerativeModel('gemini-2.0-flash-001')
    print("Modelo Gemini inicializado.   1")
    # Genera la respuesta usando el prompt
    response = model.generate_content(custom_prompt)

    # Extrae y retorna el contenido formateado
    return extract_course_content(response)

In [ ]:

class CourseFileParser:
    """Parse course content from various file formats (PDF, DOCX, TXT)."""

    @staticmethod
    def parse_file(file_path):
        """Parse a file based on its extension."""
        import os
        import logging
        from pathlib import Path

        file_path = Path(file_path)
        ext = file_path.suffix.lower()

        if ext == '.txt':
            with open(file_path, 'r', encoding='utf-8') as f:
                return f.read()
        elif ext == '.docx':
            try:
                import docx
                doc = docx.Document(file_path)
                content = '\n'.join([para.text for para in doc.paragraphs])
                return content
            except Exception as e:
                logging.error(f"Error parsing DOCX file: {e}")
                return None
        elif ext == '.pdf':
            try:
                # First attempt with PyPDF2
                try:
                    import PyPDF2
                    with open(file_path, 'rb') as f:
                        reader = PyPDF2.PdfReader(f)
                        content = ""
                        for page_num in range(len(reader.pages)):
                            page = reader.pages[page_num]
                            text = page.extract_text()
                            if text:
                                content += text + "\n\n"

                        if content.strip():  # If we got content, return it
                            return content
                except Exception as e:
                    logging.warning(f"PyPDF2 failed: {e}, trying pdfplumber...")

                # If PyPDF2 fails or returns empty content, try pdfplumber
                import pdfplumber
                content = ""
                with pdfplumber.open(file_path) as pdf:
                    for page in pdf.pages:
                        text = page.extract_text()
                        if text:
                            content += text + "\n\n"

                if not content.strip():
                    # If both methods fail to extract text, PDF might be scanned
                    logging.warning("PDF may be scanned/image-based. Consider OCR.")

                return content
            except Exception as e:
                logging.error(f"Error parsing PDF file: {e}")
                return None
        else:
            logging.error(f"Unsupported file format: {ext}")
            return None

    @staticmethod
    def upload_and_parse():
        """Upload and parse a file directly in Colab."""
        from google.colab import files
        import io
        import os
        import logging

        print("Por favor, sube un archivo de curso (PDF, DOCX o TXT):")
        uploaded = files.upload()

        for filename, content in uploaded.items():
            temp_path = f"/tmp/{filename}"
            with open(temp_path, 'wb') as f:
                f.write(content)

            parsed_content = CourseFileParser.parse_file(temp_path)
            os.remove(temp_path)  # Limpiar archivos temporales
            custom_prompt = """
            Convierte el siguiente contenido del curso  en un string  que  siguiendo EXACTAMENTE el siguiente formato:



            Contenido del curso: [Nombre del curso]
            Unidades:
            1. [Título de la unidad 1]
            2. [Título de la unidad 2]
            3. [Título de la unidad 3]

            Autor: [Nombre del autor]
            [Cargo o posición]
            [Departamento o información adicional]
            Universidad Nacional de Colombia


            En la conversión, reemplaza los campos entre corchetes con la información correspondiente del PDF (por ejemplo, para el curso "Análisis y Diseño de Algoritmos (2024-2)", ubica de forma coherente los objetivos, requisitos, evaluación, metodología y demás datos).
            """ + parsed_content
            if parsed_content and parsed_content.strip():
                print(f"Archivo {filename} procesado exitosamente.")

                new_parsed = generate_course_content(custom_prompt)
                return new_parsed
            else:
                print(f"Error al procesar {filename} o el archivo no contiene texto extraíble.")
                return None



In [ ]:
class ContentEvaluator:
    """Evaluates the quality of generated educational content."""

    def __init__(self, llm_client):
        self.llm_client = llm_client
        self.evaluation_templates = {
            "relevance": """
[INSTRUCCIÓN]
Actúa como un experto en evaluación educativa. Evalúa la relevancia y calidad del siguiente contenido educativo con respecto al tema "{topic}" y los objetivos de aprendizaje: "{objectives}".

[CONTENIDO A EVALUAR]
{content_sample}

[CRITERIOS DE EVALUACIÓN]
1. Relevancia temática (1-10): ¿El contenido aborda directamente el tema especificado?
2. Alineación con objetivos (1-10): ¿El contenido permite alcanzar los objetivos de aprendizaje?
3. Precisión académica (1-10): ¿Los conceptos, definiciones y explicaciones son precisos?
4. Profundidad adecuada (1-10): ¿El contenido tiene la profundidad apropiada para el nivel académico?
5. Organización y estructura (1-10): ¿El contenido está bien organizado y estructurado?

[FORMATO REQUERIDO]
Proporciona una puntuación de 1-10 para cada criterio, seguida de una breve justificación.
Al final, calcula una puntuación global (promedio) y ofrece 2-3 sugerencias específicas de mejora.
Tu evaluación debe ser rigurosa, objetiva y constructiva.
""",
            "readability": """
[INSTRUCCIÓN]
Actúa como un experto en comunicación educativa. Evalúa la legibilidad y claridad pedagógica del siguiente contenido educativo para estudiantes de nivel {course_level}.

[CONTENIDO A EVALUAR]
{content_sample}

[CRITERIOS DE EVALUACIÓN]
1. Claridad explicativa (1-10): ¿Las explicaciones son claras y comprensibles?
2. Lenguaje apropiado (1-10): ¿El lenguaje es apropiado para el nivel académico?
3. Ejemplificación (1-10): ¿Se utilizan ejemplos efectivos para ilustrar conceptos?
4. Recursos visuales/organizativos (1-10): ¿El contenido utiliza recursos para facilitar la comprensión?
5. Accesibilidad cognitiva (1-10): ¿El contenido es accesible para diferentes estilos de aprendizaje?

[FORMATO REQUERIDO]
Proporciona una puntuación de 1-10 para cada criterio, seguida de una breve justificación.
Al final, calcula una puntuación global (promedio) y ofrece 2-3 sugerencias específicas de mejora.
Tu evaluación debe centrarse en aspectos pedagógicos y comunicativos.
""",
            "terminology": """
[INSTRUCCIÓN]
Actúa como un experto lingüista especializado en {subject_area}. Evalúa el uso apropiado de terminología especializada en el siguiente contenido educativo.

[CONTENIDO A EVALUAR]
{content_sample}

[CRITERIOS DE EVALUACIÓN]
1. Precisión terminológica (1-10): ¿Los términos técnicos se utilizan con precisión?
2. Consistencia (1-10): ¿La terminología se utiliza de manera consistente en todo el contenido?
3. Explicación de términos (1-10): ¿Los términos técnicos se introducen y explican adecuadamente?
4. Densidad terminológica (1-10): ¿La densidad de términos técnicos es adecuada para el nivel?
5. Contextualización (1-10): ¿Los términos se presentan en contextos relevantes?

[FORMATO REQUERIDO]
Proporciona una puntuación de 1-10 para cada criterio, seguida de una breve justificación.
Incluye una lista de 5-10 términos clave identificados en el contenido.
Al final, calcula una puntuación global (promedio) y ofrece 2-3 sugerencias específicas de mejora.
""",
            "consistency": """
[INSTRUCCIÓN]
Actúa como un editor académico experto. Evalúa la consistencia interna del siguiente contenido educativo.

[CONTENIDO A EVALUAR]
{content_sample}

[CRITERIOS DE EVALUACIÓN]
1. Consistencia conceptual (1-10): ¿Los conceptos se utilizan de manera consistente?
2. Consistencia estructural (1-10): ¿La estructura y organización es consistente?
3. Consistencia estilística (1-10): ¿El estilo de escritura es consistente?
4. Progresión lógica (1-10): ¿El contenido progresa lógicamente sin contradicciones?
5. Concordancia con objetivos (1-10): ¿El contenido es consistente con los objetivos declarados?

[FORMATO REQUERIDO]
Proporciona una puntuación de 1-10 para cada criterio, seguida de una breve justificación.
Identifica cualquier inconsistencia específica encontrada.
Al final, calcula una puntuación global (promedio) y ofrece 2-3 sugerencias específicas de mejora.
"""
        }

    def _truncate_content(self, content, max_length=4000):
        """Truncate content to avoid exceeding token limits."""
        if len(content) <= max_length:
            return content

        # Tomar primeras y últimas partes para mantener contexto
        first_part = content[:max_length//2]
        last_part = content[-max_length//2:]
        return first_part + "\n\n[...contenido omitido por brevedad...]\n\n" + last_part

    def evaluate_content(self, content, topic, objectives, material_type, course_level="pregrado", subject_area=""):
        """Evaluate content using different criteria."""
        content_sample = self._truncate_content(content)
        results = {}

        # Evaluar relevancia
        relevance_prompt = self.evaluation_templates["relevance"].format(
            topic=topic,
            objectives=objectives,
            content_sample=content_sample
        )
        relevance_result, _ = self.llm_client.generate_content(relevance_prompt)
        results["relevance"] = relevance_result

        # Evaluar legibilidad
        readability_prompt = self.evaluation_templates["readability"].format(
            course_level=course_level,
            content_sample=content_sample
        )
        readability_result, _ = self.llm_client.generate_content(readability_prompt)
        results["readability"] = readability_result

        # Evaluar terminología
        terminology_prompt = self.evaluation_templates["terminology"].format(
            subject_area=subject_area,
            content_sample=content_sample
        )
        terminology_result, _ = self.llm_client.generate_content(terminology_prompt)
        results["terminology"] = terminology_result

        # Evaluar consistencia
        consistency_prompt = self.evaluation_templates["consistency"].format(
            content_sample=content_sample
        )
        consistency_result, _ = self.llm_client.generate_content(consistency_prompt)
        results["consistency"] = consistency_result

        return results

    def generate_evaluation_report(self, evaluation_results, material_type, topic):
        """Generate a comprehensive evaluation report based on individual evaluations."""
        report_prompt = f"""
[INSTRUCCIÓN]
Actúa como un experto en evaluación educativa. Genera un informe de evaluación completo para el material de tipo "{material_type}" sobre el tema "{topic}", basado en las siguientes evaluaciones específicas:

[EVALUACIÓN DE RELEVANCIA]
{evaluation_results.get('relevance', 'No disponible')}

[EVALUACIÓN DE LEGIBILIDAD]
{evaluation_results.get('readability', 'No disponible')}

[EVALUACIÓN DE TERMINOLOGÍA]
{evaluation_results.get('terminology', 'No disponible')}

[EVALUACIÓN DE CONSISTENCIA]
{evaluation_results.get('consistency', 'No disponible')}

[FORMATO REQUERIDO]
1. Resumen Ejecutivo: Síntesis general de la calidad del material (1 párrafo)
2. Fortalezas Principales: 3-5 fortalezas clave identificadas
3. Áreas de Mejora: 3-5 áreas específicas de mejora
4. Puntuación Global: Calificación general considerando todos los criterios (escala 1-10)
5. Recomendaciones Específicas: Sugerencias concretas para mejorar el material

Tu informe debe ser claro, objetivo y orientado a mejorar la calidad educativa del material.
"""

        report, _ = self.llm_client.generate_content(report_prompt)
        return report

In [ ]:
def generate_and_evaluate_complete_materials(course_content=None, file_path=None, output_dir="output_complete",
                                            model="gemini-2.0-flash-001", output_format="markdown", api_key=None):
    """Generate and evaluate all required educational materials for a course."""
    if not api_key:
        raise ValueError("API key is required")

    # Obtener el contenido del curso
    if file_path  :
        course_content = CourseFileParser.upload_and_parse()

    elif not course_content:
        # Usar el ejemplo predeterminado
        course_content = """
Contenido del curso Cálculo diferencial.
Unidades:
1. Introducción a la derivada
2. Derivada de la función compuesta
3. Aplicaciones de la derivada en problemas de razón de cambio

Autor: Juan David Ospina Arango
Profesor Asociado
Departamento de Ciencias de la Computación y de la Decisión
Universidad Nacional de Colombia
"""

    try:
        # Inicializar cliente de Gemini
        gemini_client = GeminiClient(
            api_key=api_key,
            model=model,
            temperature=0.4,
            max_tokens=8192,
            use_cache=True
        )

        # Inicializar generador de contenido
        generator = CourseContentGenerator(
            llm_client=gemini_client,
            output_dir=output_dir
        )

        # Actualizar plantillas con las adicionales
        generator.templates.update(additional_templates)

        # Inicializar evaluador
        evaluator = ContentEvaluator(llm_client=gemini_client)

        # Analizar contenido del curso
        course_info = generator.parse_course_content(course_content)

        # Definir tipos de material a generar
        material_types = ["class_notes", "exercises", "discussion_questions",
                          "learning_objectives", "suggested_readings"]

        all_materials = {}
        all_evaluations = {}
        all_files = []

        # Generar cada tipo de material
        for material_type in material_types:
            logger.info(f"Generando material: {material_type}")

            # Generar el material
            materials = generator.generate_course_materials(
                course_info=course_info,
                material_type=material_type
            )

            all_materials[material_type] = materials

            # Guardar el material
            files = generator.save_materials(
                materials=materials,
                course_info=course_info,
                material_type=material_type,
                format=output_format
            )

            all_files.extend(files)

            # Evaluar el material
            evaluations = {}
            for unit_key, content in materials.items():
                unit_number = unit_key.split('_')[1]
                unit = next((u for u in course_info.get("units", []) if u.get("number") == unit_number), None)

                if unit:
                    # Evaluar contenido
                    topic = unit.get("name", "")
                    objectives = f"Comprender conceptos y aplicaciones de {topic}"
                    subject_area = course_info.get("title", "").split()[0]

                    eval_results = evaluator.evaluate_content(
                        content=content,
                        topic=topic,
                        objectives=objectives,
                        material_type=material_type,
                        subject_area=subject_area
                    )

                    # Generar informe de evaluación
                    eval_report = evaluator.generate_evaluation_report(
                        evaluation_results=eval_results,
                        material_type=material_type,
                        topic=topic
                    )

                    evaluations[unit_key] = {
                        "detailed_evaluations": eval_results,
                        "evaluation_report": eval_report
                    }

            all_evaluations[material_type] = evaluations

        # Guardar evaluaciones
        eval_dir = Path(output_dir) / "evaluations"
        eval_dir.mkdir(parents=True, exist_ok=True)

        for material_type, evaluations in all_evaluations.items():
            eval_file = eval_dir / f"{material_type}_evaluations.md"

            with open(eval_file, 'w', encoding='utf-8') as f:
                f.write(f"# Evaluaciones para {material_type}\n\n")

                for unit_key, eval_data in evaluations.items():
                    unit_number = unit_key.split('_')[1]
                    f.write(f"## Unidad {unit_number}\n\n")
                    f.write("### Informe de Evaluación\n\n")
                    f.write(eval_data["evaluation_report"])
                    f.write("\n\n---\n\n")

            all_files.append(eval_file)

        # Generar informe general
        summary_prompt = f"""
[INSTRUCCIÓN]
Actúa como un experto en educación. Genera un informe resumido sobre la calidad general del conjunto completo de materiales educativos generados para el curso "{course_info.get('title', '')}".

[MATERIALES GENERADOS]
Se han generado los siguientes tipos de materiales:
{', '.join(material_types)}

Para {len(course_info.get('units', []))} unidades del curso.

[FORMATO REQUERIDO]
1. Resumen Ejecutivo: Visión general de la calidad del conjunto completo de materiales
2. Fortalezas del Sistema: Aspectos positivos identificados en los materiales generados
3. Áreas de Mejora: Aspectos que podrían mejorarse en futuras iteraciones
4. Recomendaciones: Sugerencias concretas para mejorar el sistema de generación
5. Conclusión: Reflexión final sobre la utilidad y aplicabilidad de los materiales generados

Tu informe debe ser objetivo, constructivo y enfocado en el valor educativo de los materiales.
"""

        summary_report, _ = gemini_client.generate_content(summary_prompt)

        summary_file = eval_dir / "summary_report.md"
        with open(summary_file, 'w', encoding='utf-8') as f:
            f.write("# Informe General de Evaluación\n\n")
            f.write(summary_report)

        all_files.append(summary_file)

        # Estadísticas de uso
        usage_stats = gemini_client.get_usage_stats()

        stats_file = eval_dir / "usage_statistics.md"
        with open(stats_file, 'w', encoding='utf-8') as f:
            f.write("# Estadísticas de Uso de API\n\n")
            f.write(f"Total de solicitudes: {usage_stats['total_requests']}\n")
            f.write(f"Total de tokens utilizados: {usage_stats['total_tokens_used']}\n")
            f.write(f"Costo total estimado: ${usage_stats['total_cost_usd']:.4f}\n")
            f.write(f"Modelos utilizados: {', '.join(usage_stats['models_used'])}\n")
            f.write(f"Promedio de tokens por solicitud: {usage_stats['average_tokens_per_request']:.2f}\n")
            f.write(f"Costo promedio por solicitud: ${usage_stats['average_cost_per_request']:.6f}\n")

        all_files.append(stats_file)

        logger.info(f"Proceso completado. Se generaron {len(all_files)} archivos.")
        return all_materials, all_evaluations, usage_stats

    except Exception as e:
        logger.error(f"Error en la generación y evaluación de contenido: {e}")
        import traceback
        logger.error(traceback.format_exc())
        return None, None, {"error": str(e)}

In [ ]:
# Ejecutar el sistema completo

# Definir o cargar el contenido del curso
course_content = """
Contenido del curso Cálculo diferencial.
Unidades:
1. Introducción a la derivada
2. Derivada de la función compuesta
3. Aplicaciones de la derivada en problemas de razón de cambio

Autor: Juan David Ospina Arango
Profesor Asociado
Departamento de Ciencias de la Computación y de la Decisión
Universidad Nacional de Colombia
"""

# Ejecutar la generación y evaluación completa
all_materials, all_evaluations, usage_stats = generate_and_evaluate_complete_materials(
    course_content=course_content,
    file_path=" ",
    output_dir="output_completo",
    model="gemini-2.0-flash-001",  # Puedes cambiar a otros modelos disponibles
    output_format="markdown",  # o "pdf" si prefieres PDFs
    api_key=api_key  # Asegúrate de tener tu API key definida
)

# Mostrar resultados
if all_materials and all_evaluations:
    print("\n¡ÉXITO! Materiales generados y evaluados correctamente")
    print(f"Total de tipos de materiales: {len(all_materials)}")
    print(f"Total de unidades procesadas: {len(next(iter(all_materials.values())))}")
    print(f"Costo total: ${usage_stats['total_cost_usd']:.4f}")
    print(f"Revisa la carpeta 'output_completo' para ver todos los archivos generados!")

    # Descargar todos los archivos generados
    print("\nDescargando archivos generados...")
    !zip -r output_completo.zip output_completo
    from google.colab import files
    files.download('output_completo.zip')
else:
    print("\nERROR: No se pudieron generar los materiales.")
    print("Detalles del error:", usage_stats.get("error", "Error desconocido"))

Por favor, sube un archivo de curso (PDF, DOCX o TXT):


Saving 01.01 - Análisis y diseño de algoritmos (2024-2).pdf to 01.01 - Análisis y diseño de algoritmos (2024-2) (2).pdf
Archivo 01.01 - Análisis y diseño de algoritmos (2024-2) (2).pdf procesado exitosamente.
Modelo Gemini inicializado.   1

¡ÉXITO! Materiales generados y evaluados correctamente
Total de tipos de materiales: 5
Total de unidades procesadas: 3
Costo total: $0.0323
Revisa la carpeta 'output_completo' para ver todos los archivos generados!

Descargando archivos generados...
  adding: output_completo/ (stored 0%)
  adding: output_completo/discussion_questions/ (stored 0%)
  adding: output_completo/discussion_questions/20250307_015611/ (stored 0%)
  adding: output_completo/discussion_questions/20250307_015611/: Análisis y Diseño de Algoritmos (3009430)_discussion_questions.md (deflated 70%)
  adding: output_completo/discussion_questions/20250307_015611/unit_1_discussion_questions.md (deflated 63%)
  adding: output_completo/discussion_questions/20250307_015611/unit_3_discussi

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>